In [1]:
import pandas as pd

# display the csv data
df_charger = pd.read_csv("/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/3.gold/charger.csv")
df_station = pd.read_csv("/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/stations.csv")

display(df_charger)
display(df_station)

,ID,Betreiber,Straße,Hausnummer,Postleitzahl,Ort,Bundesland,Breitengrad,Längengrad,Nennleistung Ladeeinrichtung [kW],...,Anzahl Ladepunkte,Steckertypen1,P1 [kW],Steckertypen2,P2 [kW],Steckertypen3,P3 [kW],Steckertypen4,P4 [kW],ID.1
0,1,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,72535,Heroldstatt,Baden-Württemberg,48.442398,9.659075,22.0,...,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,1
1,2,smopi®,Albstraße,14,72535,Heroldstatt,Baden-Württemberg,48.449353,9.672201,22.0,...,4,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,2
2,3,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,0,72589,Westerheim,Baden-Württemberg,48.510500,9.609000,22.0,...,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,3
3,4,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,Amstetten,Baden-Württemberg,48.578534,9.874840,98.0,...,2,AC Kupplung Typ 2,43.0,"DC Kupplung Combo, DC CHAdeMO",50.0,NaN,NaN,NaN,NaN,4
4,5,SWU Energie GmbH,Sterngasse,6,88481,Balzheim,Baden-Württemberg,48.179959,10.076977,22.0,...,1,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44846,44847,Energieversorgung Apolda GmbH,Adolf-Aber-Straße,0,99510,Apolda,Thüringen,51.020983,11.510480,50.0,...,1,"DC Kupplung Combo, DC CHAdeMO",50.0,NaN,NaN,NaN,NaN,NaN,NaN,44847
44847,44848,Energieversorgung Apolda GmbH,Martinskirchplatz,0,99510,Apolda,Thüringen,51.024391,11.511938,22.0,...,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,44848
44848,44849,Energieversorgung Apolda GmbH,Schlossplatz,0,99510,Apolda,Thüringen,51.021359,11.514027,22.0,...,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,44849
44849,44850,TEAG Mobil GmbH,Sulzaer Straße,0,99518,Großheringen,Thüringen,51.107188,11.659489,30.0,...,2,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,44850


,uuid,name,brand,street,house_number,post_code,city,latitude,longitude
0,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752009,9.279039
1,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,OIL!,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900
2,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760
3,eae30bf0-21dd-48cd-a369-03234fcd57df,Tankcenter Osnabrueck,Tankcenter,Wersener Str.,70,49090,Osnabrueck,52.296679,8.000490
4,eae15ca5-73c6-4989-bcbe-026ed0dcdecc,Q1 Tankstelle,Q1,Am Töpferweg,1,06917,Jessen (Elster),51.777900,13.142500
...,...,...,...,...,...,...,...,...,...
16966,005056a9-4231-1edd-999c-6a64ec8883dc,ENI,AGIP ENI,An der A 3,NaN,56412,Heiligenroth,50.454275,7.866716
16967,29168e4c-5ae6-41a5-a695-5f69498c4f5c,"Aral Tankstelle, Plauen",ARAL,A,72,08527,Plauen,50.467308,12.185224
16968,005056a9-4231-1edd-999c-1d60105ac32b,ENI,AGIP ENI,An der A 5,NaN,76532,Baden-Baden,48.810608,8.184395
16969,ac3a61b9-4ff0-49ea-af59-7532c26c2178,"Aral Tankstelle, BAT MEDENBACH OST",ARAL,A,3,65207,Wiesbaden,50.099606,8.355211


In [2]:
# show the first row of the csv data
print("Ladesäulen-Daten:")
print(df_charger.head())

print("\nTankstellen-Daten:")
print(df_station.head())

# check and compare 
print("\nSpalten in Ladesäulen-Daten:")
print(df_charger.columns)

print("\nSpalten in Tankstellen-Daten:")
print(df_station.columns)

Ladesäulen-Daten:
   ID                     Betreiber                  Straße Hausnummer  \
0   1         Albwerk GmbH & Co. KG          Ennabeurer Weg          0   
1   2                       smopi®                Albstraße         14   
2   3         Albwerk GmbH & Co. KG  Parkplatz Campingplatz          0   
3   4  EnBW mobility+ AG und Co.KG              Hauptstraße        91c   
4   5              SWU Energie GmbH              Sterngasse          6   

   Postleitzahl          Ort         Bundesland  Breitengrad  Längengrad  \
0         72535  Heroldstatt  Baden-Württemberg    48.442398    9.659075   
1         72535  Heroldstatt  Baden-Württemberg    48.449353    9.672201   
2         72589   Westerheim  Baden-Württemberg    48.510500    9.609000   
3         73340    Amstetten  Baden-Württemberg    48.578534    9.874840   
4         88481     Balzheim  Baden-Württemberg    48.179959   10.076977   

   Nennleistung Ladeeinrichtung [kW]  ... Anzahl Ladepunkte  \
0                

In [12]:
# change the column name of the "Breitengrad" & "Längengrad" into the english terms

df_charger = df_charger.rename(columns={'Breitengrad': 'latitude', 'Längengrad': 'longitude'})

csv_filename = "/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/3.gold/charger.csv"
df_charger.to_csv(csv_filename, index=False)


In [13]:
# save as new csv file, its a combination of the public charger list and the gasstation list of germany

df_merged = pd.merge(df_charger, df_station, left_on=['latitude', 'longitude'], right_on=['latitude', 'longitude'], how='inner')

csv_filename = "/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/3.gold/charger_tankstellen.csv"
df_merged.to_csv(csv_filename, index=False)